In [1]:
import pandas as pd
import re
import sqlalchemy
import unicodedata
import difflib
import xlwings as xw
from datetime import datetime
from googletrans import Translator
from os import listdir
from os.path import isfile, join

In [2]:
def get_full_cost(x, full_cost_columns):
    cost = x[['Brutto', 'ZUS pracodawcy', 'ZUS Pracodawca', 'Skladki FGSP', 'Składki FGŚP', 'Skladki FP']].sum()
    cost = cost - x[[c for c in full_cost_columns if 'Zas' in c]].sum()

    return cost

def get_protected_sheet_data(PATH):
    wb = xw.Book(PATH)

    salaries = []
    for sheet in wb.sheets:
        print(sheet)

        data_from_sheet = sheet.range('A1').expand().options(pd.DataFrame, index=False, header=True).value

        data_from_sheet['Etat'] = data_from_sheet.apply(lambda x: x['Wymiar etatu L']/x['Wymiar etatu M'], axis=1)

        data_from_sheet.rename(columns={"Tytul": "Stanowisko",
                                        "Kwota": "Stawka",
                                        "kwota": "Stawka",
                                        "Wartość": "Stawka",
                                        "wartość": "Stawka",
                                        "ETAT": 'Etat'}, inplace=True)

        data_from_sheet['FinanceSheet'] = sheet.name

        data_from_sheet['Date'] = data_from_sheet['Data']
        full_cost_columns = data_from_sheet.columns.values
        data_from_sheet['EnovaEmployerCost'] = data_from_sheet.apply(lambda x: get_full_cost(x, full_cost_columns), axis=1)

        salaries.append(data_from_sheet[['Kod', 'Pracownik', 'Stanowisko', 'Etat',
                                         'Typ umowy', 'FinanceSheet', 'Date', 'Stawka', 'EnovaEmployerCost']])

    return salaries

def etat_to_number(x):
    g = re.match('([0-9])/([0-9])', str(x).replace("'", ""))
    if g is None:
        return g
    g = g.groups()
    if len(g) == 1:
        return g[0]
    return float(g[0])/float(g[1])

def get_name(x):
    x = unicodedata.normalize('NFD', x.lower()).replace('ł', 'l').encode('ascii', 'ignore').decode()
    if '(' in x:
        full_name = re.match('(.{1,}) (.{1,}) \(.{1,}\)', x).groups()
    else:
        full_name = re.match('(.{1,}) (.{1,})', x).groups()
    return full_name[1].title() + ' ' + full_name[0].title()

In [3]:
mypath = 'C:\\Users\\anicieja\\Documents\OBJ\\PL_automation\\source_new\\total'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
onlyfiles = [file for file in onlyfiles if not '~' in file]
onlyfiles

['Total_payroll_OBSS_01-09.2018.xlsx']

In [5]:
salaries_array = []
for name in onlyfiles:
    print(name)
    salaries_array.append(pd.concat(get_protected_sheet_data('C:\\Users\\anicieja\\Documents\OBJ\\PL_automation\\source_new\\total\\' + name)))

Total_payroll_OBSS_01-09.2018.xlsx


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'Open method of Workbooks class failed', 'xlmain11.chm', 0, -2146827284), None)

In [6]:
salaries = pd.concat(salaries_array)

salaries = salaries.rename(columns={
    'Kod': 'FinanceCode',
    'Pracownik': 'FinanceName',
    'Stanowisko': 'FinancePosition',
    'Etat': 'PartTime',
    'Stawka': 'ContractValue',
    'Typ umowy': 'ContractType',
})

#salaries['PartTime'] = salaries['PartTime'].apply(etat_to_number) 
salaries = salaries.drop_duplicates()

salaries['CorrectedFinanceName'] = salaries['FinanceName'].apply(get_name)

ValueError: No objects to concatenate

In [7]:
salaries.head()

NameError: name 'salaries' is not defined

In [8]:
salaries['Date'] = salaries['Date'].apply(lambda x: x.replace(day=1))
salaries.groupby(['Date']).count()

NameError: name 'salaries' is not defined

In [9]:
salaries = salaries[~salaries['FinancePosition'].isnull()]
columns = ['FinanceCode', 'FinanceName', 'FinancePosition', 'PartTime', 'ContractType', 'FinanceSheet', 'Date', 'CorrectedFinanceName']

NameError: name 'salaries' is not defined

In [10]:
salaries = salaries.groupby(columns).sum().reset_index()

NameError: name 'salaries' is not defined

In [11]:
salaries = salaries[~(salaries['ContractValue'].isnull() & salaries['EmployerCost'].isnull())]
salaries.head()

NameError: name 'salaries' is not defined

In [12]:
salaries.head()
salaries['PartTime'] = salaries['PartTime'].fillna(1)
salaries.shape

NameError: name 'salaries' is not defined

In [13]:
salaries['Date'] = salaries['Date'].apply(lambda x: x.date())

NameError: name 'salaries' is not defined

In [14]:
DM_SERVER = "OBJPLDMSQL"
DM_DATABASE = "Structure"
DM_SCHEMA = "dbo"
DM_DB_CONNECTION = 'mssql+pyodbc://@' + DM_SERVER + '/' + DM_DATABASE + '?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes'
DM_ENGINE = sqlalchemy.create_engine(DM_DB_CONNECTION, pool_size=20, max_overflow=100)

In [15]:
employees = pd.read_sql('''
select * FROM [Structure].[dbo].[Structure_Monthly] where Date >= '{0} 00:00:00.000' and Date <= '{1} 00:00:00.000'
'''.format(salaries['Date'].min(), salaries['Date'].max()), DM_ENGINE.connect())

NameError: name 'salaries' is not defined

In [16]:
employees['Level'] = employees['Level'].replace({'': 'Regular'})

NameError: name 'employees' is not defined

In [17]:
names_changes = pd.read_sql('''
SELECT distinct concat([First_Name], ' ', [Surname]) as NameFrom
      ,concat([First_Name], ' ', [Previous_Surname]) as NameTo
FROM [Translations].[dbo].[Maiden_Name]

UNION ALL

SELECT distinct concat([First_Name], ' ', [Previous_Surname]) as NameFrom
      ,concat([First_Name], ' ', [Surname]) as NameTo
FROM [Translations].[dbo].[Maiden_Name]
''', DM_ENGINE.connect())

In [18]:
translator = Translator()
positions = salaries['FinancePosition'].unique()
translations = []
for x in positions:
    #print(x)
    try:
        translations.append(translator.translate(x.replace('/', ''), dest='en').text)
    except Exception as e:
        translations.append(x)
positions_translations = pd.DataFrame({'FinancePosition': positions, 'FinanceEnglishPosition': translations})
positions_translations.head()

NameError: name 'salaries' is not defined

In [19]:
salaries = salaries.merge(positions_translations, how='left')
salaries.shape

NameError: name 'salaries' is not defined

In [20]:
def get_matched_record(record, name_column='CorrectedFinanceName', position_column='FinanceEnglishPosition', prefix=''):
    record = record.iloc[0]
    x = record[name_column]
    #print(x)

    ratios = pd.DataFrame({'Name': employees['Name'].unique(),
                           'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in employees['Name'].unique()]
                           })

    max_ratio = ratios['Ratio'].max() - 0.1
    if max_ratio < 0.85:
        print(x)
        print(ratios['Ratio'].max())

        ratios_maiden = pd.DataFrame({'Name': names_changes['NameFrom'].tolist(),
                                      'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in names_changes['NameFrom'].tolist()]
                                      })
        max_ratio_maiden = ratios_maiden['Ratio'].max()
        matched_name_maiden = ratios_maiden[ratios_maiden['Ratio'] >= max_ratio_maiden]
        if matched_name_maiden.shape[0] >= 1 and (max_ratio_maiden - 0.1) > max_ratio:
            x = names_changes['NameTo'][names_changes['NameFrom'] == matched_name_maiden['Name'].iloc[0]].iloc[0]

            ratios = pd.DataFrame({'Name': employees['Name'].unique(),
                                   'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in employees['Name'].unique()]
                                   })

            max_ratio = ratios['Ratio'].max() - 0.1

    matched_name = ratios[ratios['Ratio'] >= max_ratio]

    x = record[position_column]
    #TODO
    #x= translator.translate(x, dest='en').text

    positions = employees[['Name', 'Position']].drop_duplicates().merge(matched_name, how='inner', on='Name')[['Name', 'Position', 'Ratio']]

    pos_ratios = pd.DataFrame({
        prefix + 'MatchedName': positions['Name'].tolist(),
        prefix + 'MatchedNameRatio': positions['Ratio'].tolist(),
        prefix + 'MatchedPosition': positions['Position'].tolist(),
        prefix + 'MatchedPositionRatio': [difflib.SequenceMatcher(None, x.lower() if x is not None else ' '
                                                                  , name.lower() if name is not None else ' ').ratio() for name in positions['Position'].tolist()]
    })
    pos_max_ratio = pos_ratios[prefix + 'MatchedPositionRatio'].max()
    matched_pos = pos_ratios[pos_ratios[prefix + 'MatchedPositionRatio'] >= pos_max_ratio]

    matched_pos = matched_pos[matched_pos[prefix + 'MatchedNameRatio'].max() == matched_pos[prefix + 'MatchedNameRatio']]

    return matched_pos

In [21]:
matched_employees = salaries.groupby(['CorrectedFinanceName', 'FinanceEnglishPosition']).apply(get_matched_record).reset_index()

NameError: name 'salaries' is not defined

In [22]:
matched_employees = matched_employees.drop(['level_2'], axis=1)

NameError: name 'matched_employees' is not defined

In [23]:
merged_salary = salaries.merge(matched_employees, how="outer")
merged_salary.shape

NameError: name 'salaries' is not defined

In [24]:
merged_salary['Ratio'] = merged_salary['MatchedNameRatio'] * merged_salary['MatchedPositionRatio']

NameError: name 'merged_salary' is not defined

In [25]:
import matplotlib.pyplot as plt
%matplotlib inline

merged_salary['Ratio'][~merged_salary['MatchedPosition'].isnull()].hist()

NameError: name 'merged_salary' is not defined

In [26]:
merged_salary[(~merged_salary['MatchedPosition'].isnull()) & (merged_salary['Ratio'] < 0.2)][['FinanceName', 'FinancePosition', 'MatchedPosition']].drop_duplicates()

NameError: name 'merged_salary' is not defined

In [27]:
employees = employees[['Name', 'Position', 'Tribe', 'Subtribe', 'Guild', 'Subguild', 'Office', 'Level', 'values',
                       'LeaveDate', 'EmploymentDate', 'Management', 'Date', 'Backoffice', 'LastWorkingDate', 'Sex']]

NameError: name 'employees' is not defined

In [28]:
employees['Date'] = employees['Date'].apply(lambda x : x.date())

NameError: name 'employees' is not defined

In [29]:
merged_salary['Date'] = pd.to_datetime(merged_salary['Date'])

NameError: name 'merged_salary' is not defined

In [30]:
merged_salary['Date'] = merged_salary['Date'].apply(lambda x : x.date())

NameError: name 'merged_salary' is not defined

In [31]:
merged_salary = merged_salary.merge(employees, how="outer", left_on=['MatchedName', 'Date'], right_on=['Name', 'Date'])

NameError: name 'merged_salary' is not defined

In [32]:
merged_salary.head()
salaries.shape[0], merged_salary.shape[0]

NameError: name 'merged_salary' is not defined

In [33]:
merged_salary['Date'].unique()

NameError: name 'merged_salary' is not defined

In [34]:
count = merged_salary.groupby(['Name', 'Date']).apply(lambda x: pd.DataFrame({'Count': [x.shape[0]]}))
count[count['Count'] > 1]

NameError: name 'merged_salary' is not defined

In [35]:
merged_salary.head()

NameError: name 'merged_salary' is not defined

In [36]:
merged_salary[(merged_salary['Name'] == 'Monika Grzech')]['Ratio'].head()

NameError: name 'merged_salary' is not defined

In [37]:
merged_salary['ContractType'].unique()

NameError: name 'merged_salary' is not defined

In [38]:
merged_salary.head()

NameError: name 'merged_salary' is not defined